In [18]:
import numpy as np
import os
import string
import random
import csv
from collections import defaultdict
import heapq
import math
import hashlib
from itertools import chain

# Core Functions

In [19]:
def hash_funtion(key):
    
    return  '{0:016b}'.format(key)

class Bucket:
    
    def __init__(self,local_depth,index,empty_spaces,id):
        
        self.id = id
        self.local_depth = local_depth
        self.index = index
        self.empty_spaces = empty_spaces

class Directory:
    
    def  __init__(self,global_depth,directory_records):
        
        self.global_depth = global_depth,
        self.directory_records = directory_records

class DirectoryRecord:
    
    def __init__(self,bucket, hash_prefix):
        
        self.hash_prefix = hash_prefix
        self.value = bucket

# Hyper Parameters

In [20]:
bucket_capacity = 2
bucket_number = 3
global_depth = 1

In [21]:
# Initialization of buckets
bucket1 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 1)
bucket2 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 2)

# Initialization of directory
directory_records = list()  

directory_records.append(DirectoryRecord(hash_prefix = 0, bucket = bucket1))
directory_records.append(DirectoryRecord(hash_prefix = 1, bucket = bucket2))

directory = Directory(global_depth = 1, directory_records = directory_records)

In [22]:
directory_records

# Insertion Algorithm

In [23]:
def insert(index):
    
    global directory
    global bucket_number
    
    t_id = index[0]
    hash_key = hash_funtion(int(t_id))
    
    hash_prefix = int(hash_key[-directory.global_depth[0]:], 2)

    bucket = directory.directory_records[hash_prefix].value
    bucket.index.append(index)
    bucket.empty_spaces = int(bucket.empty_spaces)-1

    if(bucket.empty_spaces < 0):

        tempopary_memory = bucket.index    
        bucket.empty_spaces = bucket_capacity
        bucket.index = []

        if (directory.global_depth[0] > bucket.local_depth):

            # NUMBER OF LINKED BUCKETS
            number_of_links = 2**(directory.global_depth[0] - bucket.local_depth)
            bucket.local_depth = bucket.local_depth + 1
            number_of_modify_links = number_of_links/2 

            new_bucket = Bucket(local_depth = bucket.local_depth, index=[], empty_spaces = bucket_capacity, id = bucket_number)

            for directory_record in directory.directory_records:

                if(directory_record.value == bucket):
                    if(number_of_modify_links != 0):
                        number_of_modify_links = number_of_modify_links - 1
                    else:
                        directory_record.value = new_bucket
                        bucket_number = bucket_number + 1

            for i in range(len(tempopary_memory)):
                insert(tempopary_memory[i])
                

        elif (directory.global_depth[0] == bucket.local_depth):
            
            new_directory_len = 2 * len(directory.directory_records)
            new_directory_records = []

            for directory_record_number in range(new_directory_len):
                new_directory_records.append(DirectoryRecord(hash_prefix=directory_record_number,bucket=Bucket(local_depth=1,index=[],empty_spaces=bucket_capacity,id=bucket_number)))
                bucket_number = bucket_number + 1
            
            new_directory = Directory(global_depth=directory.global_depth[0]+1,directory_records=new_directory_records)

            # REHASING

            for directory_record in directory.directory_records:
                haskey1 = '0'+hash_funtion(directory_record.hash_prefix)
                haskey2 = '1'+hash_funtion(directory_record.hash_prefix)
                new_index1 = int(haskey1[-directory.global_depth[0]:],2)
                new_index2 = int(haskey2[-directory.global_depth[0]:],2)

                new_directory.directory_records[new_index1].value = directory_record.value
                new_directory.directory_records[new_index2].value = directory_record.value

            directory= new_directory

            for i in range(len(tempopary_memory)):

                insert(tempopary_memory[i],lock)
    


# Main

In [24]:
t_id = 0
t_amount = 100
u_name = 'David'

insert([t_id, t_amount, u_name])

# Task 2 

In [25]:
class Bucket:
    _id_counter = 0

    def __init__(self, local_depth):
        self.id = Bucket._id_counter
        Bucket._id_counter += 1
        self.local_depth = local_depth
        self.data = []

    def is_full(self):
        return len(self.data) >= 3  # Bucket size limit is 3

    def __str__(self):
        return f"[B{self.id} depth={self.local_depth}] {self.data}"


class Directory:
    def __init__(self):
        self.global_depth = 1
        self.size = 2 ** self.global_depth
        self.buckets = {}
        self.directory = []

        # Initialize two buckets
        b0 = Bucket(local_depth=1)
        b1 = Bucket(local_depth=1)
        self.buckets[b0.id] = b0
        self.buckets[b1.id] = b1
        self.directory = [b0.id, b1.id]

    def _calculate_ascii_hash(self, value):
        "Calculate the sum of ASCII values of the string"
        str_val = str(value)
        ascii_sum = sum(ord(char) for char in str_val)
        return ascii_sum

    def get_bucket_index(self, hash_value):
        """Get directory index"""
        return hash_value & ((1 << self.global_depth) - 1)

    def get_bucket(self, h):
        bucket_id = self.directory[h]
        return self.buckets[bucket_id]

    def double_directory(self):
        # Double the directory
        new_directory = self.directory * 2
        self.directory = new_directory
        self.global_depth += 1
        self.size = len(self.directory)
        print(f"  >> Directory doubled to size {self.size}, global depth now {self.global_depth}")
        return True

    def can_split_bucket(self, bucket):
        "Check if bucket can be split by deeper hash"
        if len(bucket.data) < 2:
            return False

        # Check if values have different prefixes at next depth
        new_depth = bucket.local_depth + 1
        mask = (1 << new_depth) - 1
        prefixes = set()

        for val in bucket.data:
            h = self._calculate_ascii_hash(val)
            prefix = h & mask
            prefixes.add(prefix)

            # If different prefixes found, split is possible
            if len(prefixes) > 1:
                return True

        return False  # All values share the same prefix

    def split_bucket(self, bucket_id):
        old_bucket = self.buckets[bucket_id]

        # Check if split is possible
        if not self.can_split_bucket(old_bucket):
            print(f"  !! Cannot split B{old_bucket.id} - all keys have same prefix at next depth")
            return False

        old_depth = old_bucket.local_depth
        old_bucket.local_depth += 1

        # Double directory if needed
        if old_bucket.local_depth > self.global_depth:
            self.double_directory()

        # Create new bucket
        new_bucket = Bucket(local_depth=old_bucket.local_depth)
        self.buckets[new_bucket.id] = new_bucket

        # Update directory pointers
        high_bit = 1 << (old_bucket.local_depth - 1)
        for i in range(len(self.directory)):
            if self.directory[i] == bucket_id:
                # Use high bit to decide bucket
                if i & high_bit:
                    self.directory[i] = new_bucket.id

        # Re-distribute old data
        temp = old_bucket.data[:]
        old_bucket.data.clear()

        for val in temp:
            h = self._calculate_ascii_hash(val)
            # Use new depth to get bucket
            bucket_index = h & ((1 << self.global_depth) - 1)
            b = self.get_bucket(bucket_index)
            b.data.append(val)

        print(f"  >> Split B{bucket_id} into B{bucket_id} and B{new_bucket.id}")
        return True

    def insert(self, value):
        # Compute ASCII hash
        h = self.get_bucket_index(self._calculate_ascii_hash(value))
        bucket = self.get_bucket(h)
        str_val = str(value)
        ascii_sum = self._calculate_ascii_hash(value)
        print(f"  Inserting {value} ('{str_val}'→ASCII:{ascii_sum}) → index:{h}")

        if value in bucket.data:
            print(f"  !! Duplicate value {value} ignored.")
            return

        # If bucket full, try to split
        if bucket.is_full():
            print(f"  !! Bucket B{bucket.id} full. Attempting to split...")
            if not self.split_bucket(bucket.id):
                print(f"  !! ERROR: Cannot insert {value}. Bucket B{bucket.id} is full and cannot be split.")
                return

            else:
                # Get bucket again after split
                h = self.get_bucket_index(self._calculate_ascii_hash(value))
                bucket = self.get_bucket(h)

        # Insert value
        bucket.data.append(value)
        print(f"  >> Inserted {value} into B{bucket.id}")

    def __str__(self):
        result = f"\n--- Directory (global depth = {self.global_depth}, size = {self.size}) ---\n"
        seen_buckets = set()

        # Show directory entries
        for i, b_id in enumerate(self.directory):
            # Show binary index
            binary_index = bin(i)[2:].zfill(self.global_depth)[-self.global_depth:]

            if b_id not in seen_buckets:
                result += f"  {binary_index} → {self.buckets[b_id]}\n"
                seen_buckets.add(b_id)
            else:
                # Reference to existing bucket
                result += f"  {binary_index} → (ref:B{b_id})\n"

        # Show all buckets
        result += "\nAll Buckets:\n"
        for bucket_id, bucket in self.buckets.items():
            if bucket_id not in seen_buckets:
                result += f"  {bucket}\n"

        return result


def main():
    values = [16, 22, 26, 20, 3, 1, 12, 91, 28, 26, 47, 11, 13, 19, 38, 47, 46]
    directory = Directory()

    for i, val in enumerate(values):
        print(f"\nStep {i + 1}: Insert {val}")
        directory.insert(val)
        print(directory)


if __name__ == "__main__":
    main()



Step 1: Insert 16
  Inserting 16 ('16'→ASCII:103) → index:1
  >> Inserted 16 into B1

--- Directory (global depth = 1, size = 2) ---
  0 → [B0 depth=1] []
  1 → [B1 depth=1] [16]

All Buckets:


Step 2: Insert 22
  Inserting 22 ('22'→ASCII:100) → index:0
  >> Inserted 22 into B0

--- Directory (global depth = 1, size = 2) ---
  0 → [B0 depth=1] [22]
  1 → [B1 depth=1] [16]

All Buckets:


Step 3: Insert 26
  Inserting 26 ('26'→ASCII:104) → index:0
  >> Inserted 26 into B0

--- Directory (global depth = 1, size = 2) ---
  0 → [B0 depth=1] [22, 26]
  1 → [B1 depth=1] [16]

All Buckets:


Step 4: Insert 20
  Inserting 20 ('20'→ASCII:98) → index:0
  >> Inserted 20 into B0

--- Directory (global depth = 1, size = 2) ---
  0 → [B0 depth=1] [22, 26, 20]
  1 → [B1 depth=1] [16]

All Buckets:


Step 5: Insert 3
  Inserting 3 ('3'→ASCII:51) → index:1
  >> Inserted 3 into B1

--- Directory (global depth = 1, size = 2) ---
  0 → [B0 depth=1] [22, 26, 20]
  1 → [B1 depth=1] [16, 3]

All Buckets:



# Task 3

In [26]:
# Core Classes and Functions
class Bucket:
    def __init__(self, local_depth, bucket_id):
        self.local_depth = local_depth  # Number of bits used for bucket addressing
        self.data = []                  # Values stored in this bucket
        self.id = bucket_id             # Unique identifier for the bucket
    
    def is_full(self, capacity):
        return len(self.data) >= capacity  #Check if bucket has reached its capacity
    
    def __str__(self):
        return f"B{self.id}(ld={self.local_depth}): {sorted(self.data)}"

class Directory:
     #Manages the directory and buckets of the extendible hash table
        
    def __init__(self, global_depth):
        self.global_depth = global_depth          # Number of bits used for directory addressing
        self.size = 2 ** global_depth             # Current directory size
        # Initialize buckets with unique IDs
        self.buckets = [Bucket(global_depth, i) for i in range(self.size)]
        self.next_bucket_id = self.size           # Track next available bucket ID
    
    def get_bucket(self, hash_value):
        index = hash_value % self.size  # Determine bucket index
        return self.buckets[index]
    
    def double_directory(self):
        print(" Doubling directory size")
        self.global_depth += 1
        new_size = 2 ** self.global_depth
        
        # Create new directory with doubled size
        new_buckets = []
        for i in range(new_size):
            # Point to existing buckets (copy pointers)
            old_index = i % (new_size // 2)
            new_buckets.append(self.buckets[old_index])
        
        self.size = new_size
        self.buckets = new_buckets
    
    def split_bucket(self, bucket_idx):
        old_bucket = self.buckets[bucket_idx]
        print(f" Splitting bucket B{old_bucket.id}")
        
        # Create new bucket with incremented local depth
        new_bucket = Bucket(old_bucket.local_depth + 1, self.next_bucket_id)
        self.next_bucket_id += 1
        
        # Update local depths
        old_bucket.local_depth += 1
        new_bucket.local_depth = old_bucket.local_depth
        
        # Update directory pointers using high bit
        high_bit_mask = 1 << (old_bucket.local_depth - 1)
        for i in range(self.size):
            if self.buckets[i] == old_bucket and (i & high_bit_mask):
                self.buckets[i] = new_bucket
        
        return new_bucket

    def __str__(self):
        result = f"Global Depth: {self.global_depth}\nDirectory Size: {self.size}\n"
        for i, bucket in enumerate(self.buckets):
            result += f"{i} -> {bucket}\n"
        return result

def hash_function(value):
    # Convert to string and sum ASCII values of characters
    return sum(ord(c) for c in str(value))

# Hyper Parameters
BUCKET_CAPACITY = 3
INITIAL_GLOBAL_DEPTH = 1

# Insertion Algorithm
def insert(directory, value):
    # Compute hash value
    hash_value = hash_function(value)
    bucket = directory.get_bucket(hash_value)
    
    print(f"\nInserting {value} (ASCII sum={hash_value})")
    
    # Check for duplicates
    if value in bucket.data:
        print(f"Duplicate value {value} ignored")
        print(directory)
        return
    
    # If bucket has space, insert directly
    if len(bucket.data) < BUCKET_CAPACITY:
        bucket.data.append(value)
        print(f"Added to B{bucket.id}")
        print(directory)
        return
    
    print(f"Bucket B{bucket.id} overflow! Splitting required")
    
    # Check if directory needs expansion
    if bucket.local_depth == directory.global_depth:
        directory.double_directory()
        # Recompute bucket after directory expansion
        hash_value = hash_function(value)
        bucket = directory.get_bucket(hash_value)
    
    # Split the overflowing bucket
    new_bucket = directory.split_bucket(bucket.id)
    print(f"Created new bucket B{new_bucket.id}")
    
    # Redistribute data from old bucket
    all_data = bucket.data + [value]
    bucket.data = []
    new_bucket.data = []
    
    # Reinsert all values
    for val in all_data:
        val_hash = hash_function(val)
        target_bucket = directory.get_bucket(val_hash)
        target_bucket.data.append(val)
    
    print(directory)

# Deletion Functionality
def delete(directory, value):
    hash_value = hash_function(value)
    bucket = directory.get_bucket(hash_value)
    
    print(f"\nDeleting {value} (ASCII sum={hash_value})")
    
    if value in bucket.data:
        bucket.data.remove(value)
        print(f"Removed from B{bucket.id}")
    else:
        print(f"Value {value} not found")
    
    print(directory)

# Statistics Function
def get_bucket_statistics(directory):
    """Calculate statistics about bucket utilization"""
    stats = {
        'total_buckets': len(set(directory.buckets)),
        'empty_buckets': 0,
        'average_utilization': 0.0,
        'min_utilization': BUCKET_CAPACITY,
        'max_utilization': 0
    }
    
    total_utilization = 0
    unique_buckets = set()
    
    for bucket in directory.buckets:
        # Only count each bucket once
        if id(bucket) not in unique_buckets:
            unique_buckets.add(id(bucket))
            utilization = len(bucket.data) / BUCKET_CAPACITY
            total_utilization += utilization
            
            if len(bucket.data) == 0:
                stats['empty_buckets'] += 1
            
            stats['min_utilization'] = min(stats['min_utilization'], utilization)
            stats['max_utilization'] = max(stats['max_utilization'], utilization)
    
    stats['average_utilization'] = total_utilization / stats['total_buckets'] if stats['total_buckets'] > 0 else 0
    return stats

# File Loading Function (New Feature)
def load_from_file(directory, filename):
    print(f"\nLoading data from {filename}")
    try:
        with open(filename, 'r') as file:
            for line in file:
                values = line.strip().split(',')
                for value in values:
                    if value:  # Skip empty strings
                        insert(directory, int(value))
    except FileNotFoundError:
        print(f"Error: File {filename} not found")

In [27]:
def main():
    print("===== EXTENDIBLE HASHING =====")
    
    # Case 1: Basic insertion sequence
    print("\n CASE 1: Basic Insertion")
    dir1 = Directory(INITIAL_GLOBAL_DEPTH)
    values = [16, 22, 26, 20, 3, 1, 12, 91, 28, 26, 47, 11, 13, 19, 38, 47, 46]
    for value in values:
        insert(dir1, value)
        print("=" * 50)
    
    # Case 2: Random data generation
    print("\n CASE 2: Random Data Generation")
    import random
    dir2 = Directory(INITIAL_GLOBAL_DEPTH)
    for _ in range(20):
        value = random.randint(1, 100)
        insert(dir2, value)
        print("=" * 50)
    
    # Case 3: Edge cases
    print("\n CASE 3: Edge Cases")
    dir3 = Directory(INITIAL_GLOBAL_DEPTH)
    
    # Insert duplicate
    insert(dir3, 10)
    insert(dir3, 10)  # Duplicate
    
    # Insert empty (should do nothing)
    # insert(dir3, None)  # Uncomment to test
    
    # Insert very large number
    insert(dir3, 999999)
    
    # Insert negative number
    insert(dir3, -5)
    
    # Case 4: Load from file
    print("\n CASE 4: Load from File")
    
    # Create test data file
    with open('test_data.txt', 'w') as f:
        f.write("5,10,15,20,25,30,35,40,45,50")
    
    dir4 = Directory(INITIAL_GLOBAL_DEPTH)
    load_from_file(dir4, 'test_data.txt')
    
    # Case 5: Deletion and statistics
    print("\n CASE 5: Deletion and Statistics")
    dir5 = Directory(INITIAL_GLOBAL_DEPTH)
    for value in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
        insert(dir5, value)
    
    stats_before = get_bucket_statistics(dir5)
    print("\nStatistics before deletion:")
    print(stats_before)
    
    delete(dir5, 20)
    delete(dir5, 40)
    delete(dir5, 99)  # Not found
    
    stats_after = get_bucket_statistics(dir5)
    print("\nStatistics after deletion:")
    print(stats_after)

if __name__ == "__main__":
    main()

===== EXTENDIBLE HASHING =====

 CASE 1: Basic Insertion

Inserting 16 (ASCII sum=103)
Added to B1
Global Depth: 1
Directory Size: 2
0 -> B0(ld=1): []
1 -> B1(ld=1): [16]


Inserting 22 (ASCII sum=100)
Added to B0
Global Depth: 1
Directory Size: 2
0 -> B0(ld=1): [22]
1 -> B1(ld=1): [16]


Inserting 26 (ASCII sum=104)
Added to B0
Global Depth: 1
Directory Size: 2
0 -> B0(ld=1): [22, 26]
1 -> B1(ld=1): [16]


Inserting 20 (ASCII sum=98)
Added to B0
Global Depth: 1
Directory Size: 2
0 -> B0(ld=1): [20, 22, 26]
1 -> B1(ld=1): [16]


Inserting 3 (ASCII sum=51)
Added to B1
Global Depth: 1
Directory Size: 2
0 -> B0(ld=1): [20, 22, 26]
1 -> B1(ld=1): [3, 16]


Inserting 1 (ASCII sum=49)
Added to B1
Global Depth: 1
Directory Size: 2
0 -> B0(ld=1): [20, 22, 26]
1 -> B1(ld=1): [1, 3, 16]


Inserting 12 (ASCII sum=99)
Bucket B1 overflow! Splitting required
 Doubling directory size
 Splitting bucket B1
Created new bucket B2
Global Depth: 2
Directory Size: 4
0 -> B0(ld=1): [20, 22, 26]
1 -> B1(ld=2)

# Task 4

In [29]:
class ExternalMergeSort:
    def __init__(self, total_pages=100, page_size=4, buffer_size=3):
        self.total_pages = total_pages
        self.page_size = page_size
        self.buffer_size = buffer_size
        self.io_count = 0
        self.pass_count = 0
        self.data = []
        self.sorted_pages = []  # Store final sorted pages
    
    def generate_data(self):
        print(f"Generating {self.total_pages} pages:")
        self.data = [
            sorted(random.sample(range(1, 1000), self.page_size)) 
            for _ in range(self.total_pages)
        ]
        # Display all pages
        for i, page in enumerate(self.data):
            print(f"Page {i:02d}: {page}")
    
    def simulate_read(self):
        self.io_count += 1
    
    def simulate_write(self):
        self.io_count += 1

    def sort_phase(self):
        print("\n=== SORT PHASE ===")
        print("Sorting each page individually")
        # Pages are already sorted when generated, but in practice sorting might be needed
        print("Sorting complete")
    
    def merge_runs(self, runs):
        heap = []
        iterators = []
        
        # Initialize heap
        for i, run in enumerate(runs):
            # Flatten run if it's nested
            flat_run = list(chain.from_iterable(run)) if any(isinstance(item, list) for item in run) else run
            iterator = iter(flat_run)
            try:
                first_item = next(iterator)
                heapq.heappush(heap, (first_item, i, iterator))
            except StopIteration:
                pass
            iterators.append(iterator)
        
        # Merge runs
        result = []
        current_page = []
        while heap:
            val, run_idx, iterator = heapq.heappop(heap)
            current_page.append(val)
            
            # Check if page is full
            if len(current_page) >= self.page_size:
                result.append(current_page)
                self.simulate_write()  # Simulate disk write
                current_page = []
            
            # Get next item
            try:
                next_val = next(iterator)
                heapq.heappush(heap, (next_val, run_idx, iterator))
            except StopIteration:
                pass
        
        # Add last page if there are remaining items
        if current_page:
            result.append(current_page)
            self.simulate_write()
        
        return result

    def merge_phase(self):
        print("\n=== MERGE PHASE ===")
        # Initial runs: each page is a separate run
        runs = [[item for item in page] for page in self.data]
        merge_degree = self.buffer_size - 1  # Merge degree = buffer size - 1 (output page)
        
        pass_num = 0
        while len(runs) > 1:
            pass_num += 1
            new_runs = []
            print(f"\nMERGE PASS {pass_num}: Merging {len(runs)} runs with buffer size {self.buffer_size}")
            print(f"  Merge degree: {merge_degree}, Runs to process: {len(runs)}")
            print(f"  Total runs: {len(runs)}, Runs per batch: {merge_degree}, Batches: {math.ceil(len(runs)/merge_degree)}")
            
            # Process runs in batches
            for i in range(0, len(runs), merge_degree):
                batch = runs[i:i+merge_degree]
                batch_num = i//merge_degree + 1
                print(f"  Batch {batch_num}: Merging {len(batch)} runs")
                
                # Simulate read for each run in batch
                for run in batch:
                    self.simulate_read()
                
                # Merge batch
                merged_run = self.merge_runs(batch)
                new_runs.append(merged_run)
            
            runs = new_runs
            print(f"  Created {len(runs)} new runs")
        
        # Final result
        if runs:
            # Flatten final result
            self.sorted_pages = []
            for run in runs:
                if any(isinstance(item, list) for item in run):
                    # Flatten if run contains nested lists
                    flat_run = list(chain.from_iterable(run))
                else:
                    flat_run = run
                
                # Paginate the flattened run
                for i in range(0, len(flat_run), self.page_size):
                    page = flat_run[i:i+self.page_size]
                    self.sorted_pages.append(page)
            print(f"\nMerge complete in {pass_num} passes")
        else:
            self.sorted_pages = []
            print("\nMerge complete (no data)")
        
        print(f"Total I/O operations: {self.io_count}")
        
        # Display final sorted result
        print("\nFinal sorted pages:")
        for i, page in enumerate(self.sorted_pages):
            print(f"Page {i:03d}: {page}")
        
        # Verify sorting result
        if self.sorted_pages:
            all_items = [item for page in self.sorted_pages for item in page]
            is_sorted = all(all_items[i] <= all_items[i+1] for i in range(len(all_items)-1))
            
            print(f"\nVerification: Data is {'sorted' if is_sorted else 'NOT sorted'}")
            print(f"Total pages: {len(self.sorted_pages)}")
            print(f"Total items: {len(all_items)}")
        else:
            print("\nVerification: No data to verify")
        return self.sorted_pages

# Main execution
print("\n===== EXTERNAL MERGE SORT =====")
ems = ExternalMergeSort(total_pages=100, page_size=4, buffer_size=3)
ems.generate_data()
ems.sort_phase()

# Get final sorted page list
sorted_pages = ems.merge_phase()

# Display summary view
if sorted_pages:
    all_sorted_items = [item for page in sorted_pages for item in page]
    print(all_sorted_items[:400])
else:
    print("\nNo sorted data available")


===== EXTERNAL MERGE SORT =====
Generating 100 pages:
Page 00: [147, 262, 549, 799]
Page 01: [78, 483, 816, 920]
Page 02: [290, 458, 736, 757]
Page 03: [175, 335, 397, 654]
Page 04: [66, 772, 798, 915]
Page 05: [20, 536, 731, 919]
Page 06: [4, 114, 218, 786]
Page 07: [153, 553, 631, 861]
Page 08: [199, 405, 485, 636]
Page 09: [119, 130, 220, 751]
Page 10: [232, 471, 564, 796]
Page 11: [101, 425, 846, 981]
Page 12: [11, 66, 907, 977]
Page 13: [368, 549, 940, 972]
Page 14: [325, 559, 603, 784]
Page 15: [30, 84, 378, 738]
Page 16: [150, 407, 429, 826]
Page 17: [198, 334, 673, 768]
Page 18: [164, 262, 696, 725]
Page 19: [114, 525, 576, 664]
Page 20: [124, 497, 623, 798]
Page 21: [142, 276, 302, 605]
Page 22: [271, 453, 498, 944]
Page 23: [59, 618, 824, 828]
Page 24: [390, 880, 903, 988]
Page 25: [60, 155, 678, 947]
Page 26: [210, 243, 766, 882]
Page 27: [95, 466, 553, 927]
Page 28: [231, 235, 351, 615]
Page 29: [157, 435, 561, 576]
Page 30: [286, 828, 851, 911]
Page 31: [83, 213, 479, 938